In [1]:
import sys
import os

if os.path.expanduser('~') == '/Users/aaron':
    sys.path.append("/Users/aaron/Documents/github/ssdmultibox/venv/lib/python3.6/site-packages/")

import pdb
from ssdmultibox.datasets import *

np.set_printoptions(precision=15)

In [2]:
# try w/ DataLoader

In [3]:
dataset = TrainPascalFlatDataset()

In [4]:
data = dataset[1]

17 (3, 300, 300) (11640, 4) (11640,)


In [7]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, num_workers=0)

In [8]:
item = next(iter(dataloader))

12 (3, 300, 300) (11640, 4) (11640,)
17 (3, 300, 300) (11640, 4) (11640,)
23 (3, 300, 300) (11640, 4) (11640,)
26 (3, 300, 300) (11640, 4) (11640,)


In [9]:
len(item)

4

In [12]:
image_ids, ims, bbs, cats = item

In [13]:
image_ids

tensor([12, 17, 23, 26])

In [14]:
ims.shape

torch.Size([4, 3, 300, 300])

In [15]:
bbs.shape

torch.Size([4, 11640, 4])

In [16]:
cats.shape

torch.Size([4, 11640])

In [3]:
image_id, chw_im, gt_bbs, gt_cats = dataset[1]
image_id, chw_im.shape

(17, (3, 300, 300))

In [4]:
im = np.reshape(chw_im, (SIZE, SIZE, 3))
im.shape

(300, 300, 3)

In [5]:
ann = dataset.get_annotations()[image_id]
ann

{'image_path': '/Users/alelevier/data/JPEGImages/000017.jpg',
 'bbs': [[184, 61, 95, 138], [89, 77, 314, 259]],
 'cats': [14, 12]}

In [6]:
# get_stacked_gt_overlap_and_idx

In [7]:
bbs = ann['bbs']
im = open_image(ann['image_path'])
grid_size=4
aspect_ratio=(1.,1.)
overlaps = Bboxer.get_iou(bbs, im, grid_size, aspect_ratio)
overlaps.shape

(2, 16)

In [14]:
stacked_anchor_boxes = Bboxer.get_stacked_anchor_boxes(feature_maps=[2,1], aspect_ratios=lambda grid_size:[(1,1)])
stacked_intersect = Bboxer.get_stacked_intersection(bbs, im, stacked_anchor_boxes)
bbs_area = Bboxer.get_bbs_area(bbs, im)

overlaps = Bboxer.get_stacked_iou(stacked_anchor_boxes, stacked_intersect, bbs_area)
overlaps

array([[0.136393258748425, 0.087143436085416, 0.015908302303618,
        0.010568630488156, 0.073121993284493],
       [0.146321131450167, 0.157991072275379, 0.228103454417236,
        0.247705451807879, 0.460923504273504]])

In [15]:
overlaps = torch.tensor(overlaps)
overlaps

tensor([[0.1364, 0.0871, 0.0159, 0.0106, 0.0731],
        [0.1463, 0.1580, 0.2281, 0.2477, 0.4609]], dtype=torch.float64)

In [17]:
_, prior_idx = overlaps.max(1)
gt_overlap, gt_idx = overlaps.max(0)
gt_overlap[prior_idx] = 1.99
# sets the gt_idx equal to the obj_idx for each prior_idx
for i,o in enumerate(prior_idx):
    gt_idx[o] = i
gt_overlap, gt_idx

(tensor([1.9900, 0.1580, 0.2281, 0.2477, 1.9900], dtype=torch.float64),
 tensor([0, 1, 1, 1, 1]))

In [18]:
def get_stacked_gt_overlap_and_idx(stacked_anchor_boxes, stacked_intersect, bbs_area):
    overlaps = torch.tensor(
        Bboxer.get_stacked_iou(stacked_anchor_boxes, stacked_intersect, bbs_area))
    _, prior_idx = overlaps.max(1)
    gt_overlap, gt_idx = overlaps.max(0)
    gt_overlap[prior_idx] = 1.99
    # sets the gt_idx equal to the obj_idx for each prior_idx
    for i,o in enumerate(prior_idx):
        gt_idx[o] = i
    return gt_overlap, gt_idx
    
get_stacked_gt_overlap_and_idx(stacked_anchor_boxes, stacked_intersect, bbs_area)

(tensor([1.9900, 0.1580, 0.2281, 0.2477, 1.9900], dtype=torch.float64),
 tensor([0, 1, 1, 1, 1]))

In [42]:
bbs = ann['bbs']
im = open_image(ann['image_path'])

stacked_anchor_boxes = Bboxer.get_stacked_anchor_boxes(feature_maps=[2,1], aspect_ratios=lambda grid_size:[(1,1)])
stacked_intersect = Bboxer.get_stacked_intersection(bbs, im, stacked_anchor_boxes)
bbs_area = Bboxer.get_bbs_area(bbs, im)

get_stacked_gt_overlap_and_idx(stacked_anchor_boxes, stacked_intersect, bbs_area)

(tensor([1.9900, 0.1580, 0.2281, 0.2477, 1.9900], dtype=torch.float64),
 tensor([0, 1, 1, 1, 1]))

In [20]:
Bboxer.get_gt_overlap_and_idx(bbs, im)

(tensor([0.0048, 0.0763, 0.0497, 0.0064, 0.0318, 1.9900, 0.1677, 0.0430, 0.0318,
         0.1356, 0.1356, 0.0430, 0.0214, 0.0882, 0.0882, 0.0288],
        dtype=torch.float64),
 tensor([1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]))

In [21]:
# get_stacked_gt_bbs_and_cats

In [49]:
bbs = ann['bbs']
im = open_image(ann['image_path'])

stacked_anchor_boxes = Bboxer.get_stacked_anchor_boxes(feature_maps=[2,1], aspect_ratios=lambda grid_size:[(1,1)])
stacked_intersect = Bboxer.get_stacked_intersection(bbs, im, stacked_anchor_boxes)
bbs_area = Bboxer.get_bbs_area(bbs, im)

gt_overlap, gt_idx = get_stacked_gt_overlap_and_idx(stacked_anchor_boxes, stacked_intersect, bbs_area)
print(gt_overlap, gt_idx)

bbs = ann['bbs']
bbs = np.multiply(Bboxer.scaled_fastai_bbs(bbs, im), SIZE)
cats = ann['cats']
cats = np.array(cats)
gt_bbs = bbs[gt_idx]
gt_cats = cats[gt_idx]
# the previous line will set the 0 ith class as the gt, so
# set it to bg if it doesn't meet the IoU threshold
pos = gt_overlap > THRESH
try:
    neg_idx = np.nonzero(1-pos)[:,0]
except IndexError:
    # skip if no negative indexes
    pass
else:
    gt_cats[neg_idx] = 20

# gt_bbs = np.reshape(gt_bbs, (-1))

gt_bbs, gt_cats

tensor([1.9900, 0.1580, 0.2281, 0.2477, 1.9900], dtype=torch.float64) tensor([0, 1, 1, 1, 1])


(array([[ 50.27472527472527, 115.00000000000001, 163.010989010989  ,
         173.37500000000003],
        [ 63.46153846153846,  55.625           , 275.92307692307696,
         250.87500000000003],
        [ 63.46153846153846,  55.625           , 275.92307692307696,
         250.87500000000003],
        [ 63.46153846153846,  55.625           , 275.92307692307696,
         250.87500000000003],
        [ 63.46153846153846,  55.625           , 275.92307692307696,
         250.87500000000003]]), array([14, 20, 20, 20, 12]))

In [50]:
gt_bbs.shape

(5, 4)

In [46]:
dataset.categories()[14], dataset.categories()[12]

('person', 'horse')

In [13]:
stacked_anchor_boxes = Bboxer.get_stacked_anchor_boxes(feature_maps=[2,1], aspect_ratios=lambda grid_size:[(1,1)])
stacked_anchor_boxes

array([[0. , 0. , 0.5, 0.5],
       [0. , 0.5, 0.5, 1. ],
       [0.5, 0. , 1. , 0.5],
       [0.5, 0.5, 1. , 1. ],
       [0. , 0. , 1. , 1. ]])

In [46]:
anc_bbs_area = Bboxer.get_anchor_box_area(stacked_anchor_boxes)
# anc_bbs_area = np.expand_dims(anc_bbs_area, 1)
anc_bbs_area

array([0.25, 0.25, 0.25, 0.25, 1.  ])

In [18]:
bbs_area = Bboxer.get_bbs_area(bbs, im)
bbs_area

array([0.073121993284493, 0.460923504273504])

In [19]:
stacked_intersect = Bboxer.get_stacked_intersection(bbs, im, stacked_anchor_boxes)
stacked_intersect

array([[0.038782051282051, 0.090745192307692],
       [0.02590086996337 , 0.096995192307692],
       [0.005059829059829, 0.132044337606838],
       [0.003379242979243, 0.141138782051282],
       [0.073121993284493, 0.460923504273504]])

In [20]:
stacked_intersect2 = np.array([[0.038782051282051, 0.02590086996337 , 0.005059829059829,
            0.003379242979243, 0.073121993284493],
            [0.090745192307692, 0.096995192307692, 0.132044337606838,
            0.141138782051282, 0.460923504273504]])

stacked_intersect2

array([[0.038782051282051, 0.02590086996337 , 0.005059829059829,
        0.003379242979243, 0.073121993284493],
       [0.090745192307692, 0.096995192307692, 0.132044337606838,
        0.141138782051282, 0.460923504273504]])

In [23]:
anc_bbs_area + bbs_area - stacked_intersect

array([[0.284339942002442, 0.620178311965812],
       [0.297221123321123, 0.613928311965812],
       [0.318062164224664, 0.578879166666667],
       [0.31974275030525 , 0.569784722222222],
       [1.               , 1.               ]])

In [41]:
np.repeat(bbs_area, 5).reshape(2, -1)

array([[0.073121993284493, 0.073121993284493, 0.073121993284493,
        0.073121993284493, 0.073121993284493],
       [0.460923504273504, 0.460923504273504, 0.460923504273504,
        0.460923504273504, 0.460923504273504]])

In [56]:
stacked_intersect2, stacked_intersect2.shape

(array([[0.038782051282051, 0.02590086996337 , 0.005059829059829,
         0.003379242979243, 0.073121993284493],
        [0.090745192307692, 0.096995192307692, 0.132044337606838,
         0.141138782051282, 0.460923504273504]]), (2, 5))

In [57]:
union = anc_bbs_area + np.repeat(bbs_area, 5).reshape(2, -1) - stacked_intersect2
union

array([[0.284339942002442, 0.297221123321123, 0.318062164224664,
        0.31974275030525 , 1.               ],
       [0.620178311965812, 0.613928311965812, 0.578879166666666,
        0.569784722222222, 1.               ]])

In [50]:
(anc_bbs_area + np.repeat(bbs_area, 5).reshape(2, -1) - stacked_intersect2).T

array([[0.284339942002442, 0.620178311965812],
       [0.297221123321123, 0.613928311965812],
       [0.318062164224664, 0.578879166666666],
       [0.31974275030525 , 0.569784722222222],
       [1.               , 1.               ]])

In [52]:
Bboxer.get_stacked_union(stacked_anchor_boxes, stacked_intersect, bbs_area)

array([[0.284339942002442, 0.620178311965812],
       [0.297221123321123, 0.613928311965812],
       [0.318062164224664, 0.578879166666667],
       [0.31974275030525 , 0.569784722222222],
       [1.               , 1.               ]])

In [55]:
anc_bbs_area

array([0.25, 0.25, 0.25, 0.25, 1.  ])

In [58]:
# iou check

In [59]:
Bboxer.get_stacked_iou(stacked_anchor_boxes, stacked_intersect, bbs_area)

array([[0.136393258748425, 0.146321131450167],
       [0.087143436085416, 0.157991072275379],
       [0.015908302303618, 0.228103454417236],
       [0.010568630488156, 0.247705451807879],
       [0.073121993284493, 0.460923504273504]])

In [60]:
stacked_intersect2 / union

array([[0.136393258748424, 0.087143436085416, 0.015908302303618,
        0.010568630488156, 0.073121993284493],
       [0.146321131450167, 0.157991072275379, 0.228103454417237,
        0.247705451807878, 0.460923504273504]])

In [61]:
(stacked_intersect2 / union).T

array([[0.136393258748424, 0.146321131450167],
       [0.087143436085416, 0.157991072275379],
       [0.015908302303618, 0.228103454417237],
       [0.010568630488156, 0.247705451807878],
       [0.073121993284493, 0.460923504273504]])

In [40]:
np.tile(bbs_area, stacked_intersect2.shape[1]).T.reshape(stacked_intersect2.shape[0], -1) 

array([[0.073121993284493, 0.460923504273504, 0.073121993284493,
        0.460923504273504, 0.073121993284493],
       [0.460923504273504, 0.073121993284493, 0.460923504273504,
        0.073121993284493, 0.460923504273504]])

In [38]:
stacked_intersect2

array([[0.038782051282051, 0.02590086996337 , 0.005059829059829,
        0.003379242979243, 0.073121993284493],
       [0.090745192307692, 0.096995192307692, 0.132044337606838,
        0.141138782051282, 0.460923504273504]])

In [39]:
stacked_intersect2

array([[0.038782051282051, 0.02590086996337 , 0.005059829059829,
        0.003379242979243, 0.073121993284493],
       [0.090745192307692, 0.096995192307692, 0.132044337606838,
        0.141138782051282, 0.460923504273504]])

In [11]:
np.array([[0.038782051282051, 0.090745192307692],
            [0.02590086996337 , 0.096995192307692],
            [0.005059829059829, 0.132044337606838],
            [0.003379242979243, 0.141138782051282],
            [0.073121993284493, 0.460923504273504]]).T

array([[0.038782051282051, 0.02590086996337 , 0.005059829059829,
        0.003379242979243, 0.073121993284493],
       [0.090745192307692, 0.096995192307692, 0.132044337606838,
        0.141138782051282, 0.460923504273504]])

In [6]:
# get_anchor_box_area

In [9]:
sab = Bboxer.get_stacked_anchor_boxes(feature_maps=[2,1], aspect_ratios=lambda grid_size:[(1,1)])
sab

array([[0. , 0. , 0.5, 0.5],
       [0. , 0.5, 0.5, 1. ],
       [0.5, 0. , 1. , 0.5],
       [0.5, 0.5, 1. , 1. ],
       [0. , 0. , 1. , 1. ]])

In [14]:
(sab[:,2]-sab[:,0])*(sab[:,3]-sab[:,1])

array([0.25, 0.25, 0.25, 0.25, 1.  ])

In [37]:
def get_anchor_box_area(sab):
    return (sab[:,2]-sab[:,0])*(sab[:,3]-sab[:,1])

get_anchor_box_area(sab)

array([0.25, 0.25, 0.25, 0.25, 1.  ])

In [39]:
sab2 = Bboxer.get_stacked_anchor_boxes(feature_maps=[4], aspect_ratios=lambda grid_size:[(2,1)])
sab2, get_anchor_box_area(sab2)

(array([[0.   , 0.   , 0.375, 0.25 ],
        [0.   , 0.25 , 0.375, 0.5  ],
        [0.   , 0.5  , 0.375, 0.75 ],
        [0.   , 0.75 , 0.375, 1.   ],
        [0.125, 0.   , 0.625, 0.25 ],
        [0.125, 0.25 , 0.625, 0.5  ],
        [0.125, 0.5  , 0.625, 0.75 ],
        [0.125, 0.75 , 0.625, 1.   ],
        [0.375, 0.   , 0.875, 0.25 ],
        [0.375, 0.25 , 0.875, 0.5  ],
        [0.375, 0.5  , 0.875, 0.75 ],
        [0.375, 0.75 , 0.875, 1.   ],
        [0.625, 0.   , 1.   , 0.25 ],
        [0.625, 0.25 , 1.   , 0.5  ],
        [0.625, 0.5  , 1.   , 0.75 ],
        [0.625, 0.75 , 1.   , 1.   ]]),
 array([0.09375, 0.09375, 0.09375, 0.09375, 0.125  , 0.125  , 0.125  ,
        0.125  , 0.125  , 0.125  , 0.125  , 0.125  , 0.09375, 0.09375,
        0.09375, 0.09375]))

In [40]:
# use the new get_anchor_box_area to find union

In [45]:
bbs = ann['bbs']
im = open_image(ann['image_path'])
stacked_anchor_boxes = Bboxer.get_stacked_anchor_boxes(feature_maps=[2,1], aspect_ratios=lambda grid_size:[(1,1)])

stacked_intersect = Bboxer.get_stacked_intersection(bbs, im, stacked_anchor_boxes)

stacked_intersect.T.reshape(-1,2), stacked_intersect.shape

(array([[0.038782051282051, 0.090745192307692],
        [0.02590086996337 , 0.096995192307692],
        [0.005059829059829, 0.132044337606838],
        [0.003379242979243, 0.141138782051282],
        [0.073121993284493, 0.460923504273504]]), (2, 5))

In [47]:
sab, get_anchor_box_area(sab)

(array([[0. , 0. , 0.5, 0.5],
        [0. , 0.5, 0.5, 1. ],
        [0.5, 0. , 1. , 0.5],
        [0.5, 0.5, 1. , 1. ],
        [0. , 0. , 1. , 1. ]]), array([0.25, 0.25, 0.25, 0.25, 1.  ]))

In [54]:
get_anchor_box_area(sab)

array([0.25, 0.25, 0.25, 0.25, 1.  ])

In [57]:
bbs_area = Bboxer.get_bbs_area(bbs, im)
bbs_area, bbs_area.shape

(array([0.073121993284493, 0.460923504273504]), (2,))

In [60]:
np.expand_dims(get_anchor_box_area(sab), axis=1) + bbs_area

array([[0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [1.073121993284493, 1.460923504273504]])

In [61]:
np.expand_dims(get_anchor_box_area(sab), axis=1) + bbs_area - stacked_intersect.T.reshape(-1,2)

array([[0.284339942002442, 0.620178311965812],
       [0.297221123321123, 0.613928311965812],
       [0.318062164224664, 0.578879166666667],
       [0.31974275030525 , 0.569784722222222],
       [1.               , 1.               ]])

In [68]:
def get_stacked_union(stacked_anchor_boxes, stacked_intersect, bbs_area):
    anc_bbs_area = np.expand_dims(get_anchor_box_area(stacked_anchor_boxes), axis=1) 
    intersect_area = stacked_intersect.T.reshape(-1,2)
    return anc_bbs_area + bbs_area - intersect_area
    
get_stacked_union(sab, stacked_intersect, bbs_area)

array([[0.284339942002442, 0.620178311965812],
       [0.297221123321123, 0.613928311965812],
       [0.318062164224664, 0.578879166666667],
       [0.31974275030525 , 0.569784722222222],
       [1.               , 1.               ]])

In [69]:
# get_stacked_iou

In [67]:
stacked_intersect.T.reshape(-1,2)

array([[0.038782051282051, 0.090745192307692],
       [0.02590086996337 , 0.096995192307692],
       [0.005059829059829, 0.132044337606838],
       [0.003379242979243, 0.141138782051282],
       [0.073121993284493, 0.460923504273504]])

In [66]:
stacked_intersect.T.reshape(-1,2) / get_stacked_union(sab, stacked_intersect, bbs_area)

array([[0.136393258748425, 0.146321131450167],
       [0.087143436085416, 0.157991072275379],
       [0.015908302303618, 0.228103454417236],
       [0.010568630488156, 0.247705451807879],
       [0.073121993284493, 0.460923504273504]])

In [71]:
def get_stacked_iou(sab, stacked_intersect, bbs_area):
    intersect = stacked_intersect.T.reshape(-1,2)
    union = get_stacked_union(sab, stacked_intersect, bbs_area)
    return intersect / union
    
get_stacked_iou(sab, stacked_intersect, bbs_area)

array([[0.136393258748425, 0.146321131450167],
       [0.087143436085416, 0.157991072275379],
       [0.015908302303618, 0.228103454417236],
       [0.010568630488156, 0.247705451807879],
       [0.073121993284493, 0.460923504273504]])

`get_stacked_intersection` should return shape of (anc_bbs_count, bbs_count)

Added to Bboxer:

- `get_anchor_box_area`
- `get_stacked_union`
- `get_stacked_iou`

### next - test above funcs more

In [6]:
# understand how gt_overlaps and gt_idx are being set
# so it can be used with the "stacked" functions

In [19]:
bbs = ann['bbs']
im = open_image(ann['image_path'])
grid_size = 2
aspect_ratio = (1,1)

Bboxer.get_gt_overlap_and_idx(bbs, im, grid_size, aspect_ratio)

(tensor([1.9900, 0.1580, 0.2281, 1.9900], dtype=torch.float64),
 tensor([0, 1, 1, 1]))

In [20]:
overlaps = torch.tensor(Bboxer.get_iou(bbs, im, grid_size, aspect_ratio))
overlaps

tensor([[0.1364, 0.0871, 0.0159, 0.0106],
        [0.1463, 0.1580, 0.2281, 0.2477]], dtype=torch.float64)

In [21]:
overlaps.shape

torch.Size([2, 4])

In [24]:
prior_overlap, prior_idx = overlaps.max(1)
prior_overlap, prior_idx

(tensor([0.1364, 0.2477], dtype=torch.float64), tensor([0, 3]))

In [25]:
gt_overlap, gt_idx = overlaps.max(0)
gt_overlap, gt_idx

(tensor([0.1463, 0.1580, 0.2281, 0.2477], dtype=torch.float64),
 tensor([1, 1, 1, 1]))

In [33]:
gt_overlap[prior_idx] = 1.99
gt_overlap

tensor([1.9900, 0.1580, 0.2281, 1.9900], dtype=torch.float64)

In [34]:
gt_idx

tensor([1, 1, 1, 1])

In [42]:
for i,o in enumerate(prior_idx):
    print(i, o.item())
    gt_idx[o] = i
    
gt_idx

0 0
1 3


tensor([0, 1, 1, 1])

In [43]:
# get_stacked_iou

In [53]:
bbs_area = Bboxer.get_bbs_area(bbs, im)
bbs_area

array([0.073121993284493, 0.460923504273504])

In [51]:
grid_size, Bboxer.get_ancb_area(grid_size)

(2, 0.25)

In [92]:
anchor_bbs = Bboxer.stacked_anchor_boxes(feature_maps=[2,1], aspect_ratios=lambda grid_size:[(1,1)])
anchor_bbs

array([[0. , 0. , 0.5, 0.5],
       [0. , 0.5, 0.5, 1. ],
       [0.5, 0. , 1. , 0.5],
       [0.5, 0.5, 1. , 1. ],
       [0. , 0. , 1. , 1. ]])

In [84]:
bbs

[[184, 61, 95, 138], [89, 77, 314, 259]]

In [86]:
np.square(4)

16

In [87]:
np.array(bbs).shape

(2, 4)

In [ ]:
def get_stacked_anchor_boxes_area(bbs_area, feature_maps, aspect_ratios):
    aspect_ratios = aspect_ratios or cls.aspect_ratios
    boxes = None
    for i in range(len(feature_maps)):
        grid_size = feature_maps[i]
        for aspect_ratio in aspect_ratios(grid_size):
            arr = np.repeat(
                Bboxer.get_ancb_area(grid_size)*aspect_ratios[0]*aspect_ratios[1],
                np.square(grid_size)*bbs.shape[0]
            ).reshape(-1, bbs.shape[0]) + bbs_area
            if not isinstance(boxes, np.ndarray):
                boxes = arr
            else:
                boxes = np.concatenate((boxes, arr))
                
    stacked_anchor_boxes = Bboxer.stacked_anchor_boxes(feature_maps, aspect_ratios)
    stacked_intersect = Bboxer.get_stacked_intersection(bbs, im, stacked_anchor_boxes)
    intersect = stacked_intersect.T.reshape(boxes.shape[0], -1)
    
    pdb.set_trace()

    return boxes - intersect
    

bbs_area = Bboxer.get_bbs_area(bbs, im)
get_stacked_anchor_boxes_area(bbs_area, feature_maps=[2,1], aspect_ratios=lambda grid_size:[(1,1), (1,2)])

In [177]:
get_stacked_anchor_boxes_area(bbs_area, feature_maps=[2], aspect_ratios=lambda grid_size:[(1,1), (1,2)])

array([[0.284339942002442, 0.620178311965812],
       [0.297221123321123, 0.613928311965812],
       [0.318062164224664, 0.578879166666667],
       [0.31974275030525 , 0.569784722222222],
       [0.258439072039072, 0.548062927350427],
       [0.258439072039072, 0.541812927350427],
       [0.314682921245421, 0.473943269230769],
       [0.314682921245421, 0.464848824786325]])

In [175]:
ann['cats']

[14, 12]

In [176]:
dataset.categories()[14]

'person'

In [129]:
grid_size = 2

bbs = np.array(bbs)

ret = np.repeat(
    Bboxer.get_ancb_area(grid_size), np.square(grid_size)*bbs.shape[0]
).reshape(-1, bbs.shape[0]) + bbs_area
ret, ret.shape

(array([[0.323121993284493, 0.710923504273504],
        [0.323121993284493, 0.710923504273504],
        [0.323121993284493, 0.710923504273504],
        [0.323121993284493, 0.710923504273504]]), (4, 2))

In [133]:
grid_size = 1

ret2 = np.repeat(
    Bboxer.get_ancb_area(grid_size), np.square(grid_size)*bbs.shape[0]
).reshape(-1, bbs.shape[0]) + bbs_area
ret2, ret2.shape

(array([[1.073121993284493, 1.460923504273504]]), (1, 2))

In [149]:
np.square(grid_size)*bbs.shape[0]

2

In [151]:
bbs_area

array([0.073121993284493, 0.460923504273504])

In [136]:
np.concatenate((ret, ret2))

array([[0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [1.073121993284493, 1.460923504273504]])

In [144]:
stacked_anchor_boxes = Bboxer.stacked_anchor_boxes(feature_maps=[2,1], aspect_ratios=lambda grid_size:[(1,1)])
stacked_anchor_boxes, stacked_anchor_boxes.shape

(array([[0. , 0. , 0.5, 0.5],
        [0. , 0.5, 0.5, 1. ],
        [0.5, 0. , 1. , 0.5],
        [0.5, 0.5, 1. , 1. ],
        [0. , 0. , 1. , 1. ]]), (5, 4))

In [145]:
stacked_anchor_boxes = Bboxer.stacked_anchor_boxes(feature_maps=[2,1], aspect_ratios=lambda grid_size:[(1,1)])

stacked_intersect = Bboxer.get_stacked_intersection(bbs, im, stacked_anchor_boxes)

stacked_intersect, stacked_intersect.shape

(array([[0.038782051282051, 0.02590086996337 , 0.005059829059829,
         0.003379242979243, 0.073121993284493],
        [0.090745192307692, 0.096995192307692, 0.132044337606838,
         0.141138782051282, 0.460923504273504]]), (2, 5))

In [146]:
stacked_intersect.reshape(-1, 2)

array([[0.038782051282051, 0.02590086996337 ],
       [0.005059829059829, 0.003379242979243],
       [0.073121993284493, 0.090745192307692],
       [0.096995192307692, 0.132044337606838],
       [0.141138782051282, 0.460923504273504]])

In [167]:
raw_ret = np.concatenate((ret, ret2)) - stacked_intersect.T.reshape(5, -1)
raw_ret

array([[0.284339942002442, 0.620178311965812],
       [0.297221123321123, 0.613928311965812],
       [0.318062164224664, 0.578879166666667],
       [0.31974275030525 , 0.569784722222222],
       [1.               , 1.               ]])

In [168]:
np.array(
    [[0.284339942002442, 0.620178311965812],
    [0.297221123321123, 0.613928311965812],
    [0.318062164224664, 0.578879166666667],
    [0.31974275030525 , 0.569784722222222],
    [1.               , 1.               ]]) == raw_ret

array([[False,  True],
       [False,  True],
       [False, False],
       [False, False],
       [ True,  True]])

In [162]:
bbs_area

array([0.073121993284493, 0.460923504273504])

In [163]:
ret

array([[0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504]])

In [161]:
ret2

array([[1.073121993284493, 1.460923504273504]])

In [165]:
np.concatenate((ret, ret2))

array([[0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [1.073121993284493, 1.460923504273504]])

In [50]:
stacked_intersect = Bboxer.get_stacked_intersection(bbs, im, anchor_bbs)
stacked_intersect, stacked_intersect.shape

(array([[0.038782051282051, 0.02590086996337 , 0.005059829059829,
         0.003379242979243, 0.073121993284493],
        [0.090745192307692, 0.096995192307692, 0.132044337606838,
         0.141138782051282, 0.460923504273504]]), (2, 5))

In [73]:
bbs_area_union = (np.repeat(.25, 8).reshape(4,2) + bbs_area)
bbs_area_union, bbs_area_union.shape

(array([[0.323121993284493, 0.710923504273504],
        [0.323121993284493, 0.710923504273504],
        [0.323121993284493, 0.710923504273504],
        [0.323121993284493, 0.710923504273504]]), (4, 2))

In [74]:
bbs_area

array([0.073121993284493, 0.460923504273504])

In [79]:
np.tile(bbs_area, 4).reshape(4,2) + .25

array([[0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504]])

In [80]:
np.tile(bbs_area, 4).reshape(4,2) + np.repeat(.25, 8).reshape(4,2)

array([[0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504],
       [0.323121993284493, 0.710923504273504]])

In [21]:
im = open_image(ann['image_path'])
scaled_bbs = Bboxer.scaled_fastai_bbs(ann['bbs'], im=im)
scaled_bbs

array([[0.167582417582418, 0.383333333333333, 0.543369963369963,
        0.577916666666667],
       [0.211538461538462, 0.185416666666667, 0.91974358974359 ,
        0.83625          ]])

In [11]:
# plot and make sure correct

In [12]:
from ssdmultibox.plotting import show_img, draw_rect

In [25]:
# ok

# resized_im = cv2.resize(im, (SIZE, SIZE))
# ax = show_img(resized_im)
# for bb in scaled_bbs:
#     draw_rect(ax, Bboxer.fastai_bb_to_pascal_bb(bb)*SIZE)

In [30]:
anc1 = Bboxer.anchor_corners(2, (1,1))
anc1

array([[0. , 0. , 0.5, 0.5],
       [0. , 0.5, 0.5, 1. ],
       [0.5, 0. , 1. , 0.5],
       [0.5, 0.5, 1. , 1. ]])

In [32]:
anc2 = Bboxer.anchor_corners(1, (1,1))
anc2

array([[0., 0., 1., 1.]])

In [33]:
np.concatenate((anc1, anc2))

array([[0. , 0. , 0.5, 0.5],
       [0. , 0.5, 0.5, 1. ],
       [0.5, 0. , 1. , 0.5],
       [0.5, 0.5, 1. , 1. ],
       [0. , 0. , 1. , 1. ]])

In [49]:
def stacked_anchor_boxes(feature_maps=FEATURE_MAPS, aspect_ratios=None):
    aspect_ratios = aspect_ratios or cls.aspect_ratios
    boxes = None
    for i in range(len(feature_maps)):
        grid_size = feature_maps[i]
        for aspect_ratio in aspect_ratios(grid_size):
            arr = np.clip(Bboxer.anchor_corners(grid_size, aspect_ratio), 0, 1)
            if not isinstance(boxes, np.ndarray):
                boxes = arr
            else:
                boxes = np.concatenate((boxes, arr))
    return boxes

anchor_bbs = stacked_anchor_boxes(feature_maps=[2,1], aspect_ratios=lambda grid_size:[(1,1)])
anchor_bbs

array([[0. , 0. , 0.5, 0.5],
       [0. , 0.5, 0.5, 1. ],
       [0.5, 0. , 1. , 0.5],
       [0.5, 0.5, 1. , 1. ],
       [0. , 0. , 1. , 1. ]])

In [50]:
anchor_bbs.shape

(5, 4)

In [113]:
def get_stacked_intersection(bbs, im, anchor_bbs):
    bbs = Bboxer.scaled_fastai_bbs(bbs, im)
    bbs_count = anchor_bbs.shape[0]
    bbs16 = np.reshape(np.tile(bbs, bbs_count), (-1,bbs_count,4))
    intersect = np.minimum(
        np.maximum(anchor_bbs[:,:2], bbs16[:,:,:2]) - \
        np.minimum(anchor_bbs[:,2:], bbs16[:,:,2:]), 0)
    return intersect[:,:,0] * intersect[:,:,1]

anchor_bbs = stacked_anchor_boxes(
    feature_maps=[1], aspect_ratios=lambda grid_size:[(1,1)])

ann = dataset.get_annotations()[12]

get_stacked_intersection(ann['bbs'], im, anchor_bbs)

array([[0.201791531531531]])

In [117]:
ann = dataset.get_annotations()[image_id]

anchor_bbs = stacked_anchor_boxes(feature_maps=[1], aspect_ratios=lambda grid_size:[(1,1)])

stacked_intersect = get_stacked_intersection(ann['bbs'], open_image(ann['image_path']), anchor_bbs)
stacked_intersect.shape, stacked_intersect

((2, 1), array([[0.073121993284493],
        [0.460923504273504]]))

In [116]:
anchor_bbs

array([[0., 0., 1., 1.]])

In [104]:
anchor_bbs, anchor_bbs.shape

(array([[0., 0., 1., 1.]]), (1, 4))

In [98]:
anchor_bbs = stacked_anchor_boxes(feature_maps=[2,1], aspect_ratios=lambda grid_size:[(1,1)])
anchor_bbs

array([[0. , 0. , 0.5, 0.5],
       [0. , 0.5, 0.5, 1. ],
       [0.5, 0. , 1. , 0.5],
       [0.5, 0.5, 1. , 1. ],
       [0. , 0. , 1. , 1. ]])

In [100]:
stacked_intersect = get_stacked_intersection(ann['bbs'], im, anchor_bbs)
stacked_intersect.shape, stacked_intersect

((2, 5), array([[0.038782051282051, 0.02590086996337 , 0.005059829059829,
         0.003379242979243, 0.073121993284493],
        [0.090745192307692, 0.096995192307692, 0.132044337606838,
         0.141138782051282, 0.460923504273504]]))

In [110]:
ann = dataset.get_annotations()[12]
bbs = ann['bbs']
im = open_image(ann['image_path'])
anchor_bbs = stacked_anchor_boxes(
    feature_maps=[1], aspect_ratios=lambda grid_size:[(1.,1.)])

ret = get_stacked_intersection(bbs, im, anchor_bbs)
ret

array([[0.201791531531531]])

In [111]:
anchor_bbs = stacked_anchor_boxes(feature_maps=[2,1], aspect_ratios=lambda grid_size:[(1,1)])
ret = get_stacked_intersection(bbs, im, anchor_bbs)
ret

array([[0.040225225225225, 0.04206006006006 , 0.058420720720721,
        0.061085525525526, 0.201791531531531]])